In [ ]:
#this example code that i used to scrape bus details of APSRTC

import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException


driver = webdriver.Chrome()
driver.get("https://www.redbus.in/")
driver.maximize_window()
wait = WebDriverWait(driver, 10)

bus_data = []

try:
    try:
        rtc_section_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@href='https://www.redbus.in/online-booking/rtc-directory']"))).click()
    except ElementClickInterceptedException:
        print("ERROR  while clicking retc section")

    try:
        main_window = driver.current_window_handle
        for window in driver.window_handles:
            if window != main_window:
                driver.switch_to.window(window)
                break

        APSRTC_LINK = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "APSRTC"))).click()
    except ElementClickInterceptedException:
        print("ERROR while switching windows or clicking APSRTC link:")

    def collect_data():
        try:
            route_containers = driver.find_elements(By.CLASS_NAME, "route")
            for route_container in route_containers:
                title = route_container.get_attribute('title')
                href = route_container.get_attribute('href')
                route_names.append(title)
                route_links.append(href)
        except NoSuchElementException:
            print("ERROR  while collecting route data")

    def collect_route_details():
        try:
            for i in range(1, 6):
                if i > 1:
                    x_path = f'/html/body/div[1]/div/div[4]/div[12]/div[{i}]'
                    element = driver.find_element(By.XPATH, x_path)
                    driver.execute_script("arguments[0].scrollIntoView(true);", element)
                    driver.execute_script("arguments[0].click();", element)
                    time.sleep(4)
                collect_data()
        except NoSuchElementException:
            print("ERROR while collecting route details")

    def collect_bus_elements(route_name, route_link):
        try:
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'clearfix.bus-item')))
            print('found container')
            time.sleep(3)

            bus_elements = driver.find_elements(By.CLASS_NAME, 'clearfix.bus-item')

            for bus in bus_elements:
                bus_info = {}
                bus_info['Route_Name'] = route_name
                bus_info['Route_Link'] = route_link

                try:
                    bus_info['Bus_Name'] = bus.find_element(By.CLASS_NAME, "travels.lh-24.f-bold.d-color").text
                except NoSuchElementException:
                    bus_info['Bus_Name'] = np.nan

                try:
                    bus_info['Bus_Type'] = bus.find_element(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus").text
                except NoSuchElementException:
                    bus_info['Bus_Type'] = np.nan

                try:
                    bus_info['Departing_Time'] = bus.find_element(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold").text
                except NoSuchElementException:
                    bus_info['Departing_Time'] = np.nan

                try:
                    bus_info['Duration'] = bus.find_element(By.CLASS_NAME, "dur.l-color.lh-24").text
                except NoSuchElementException:
                    bus_info['Duration'] = np.nan

                try:
                    bus_info['Reaching_Time'] = bus.find_element(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline").text
                except NoSuchElementException:
                    bus_info['Reaching_Time'] = np.nan

                try:
                    bus_info['Star_Rating'] = bus.find_element(By.CLASS_NAME, "rating-sec.lh-24").text
                except NoSuchElementException:
                    bus_info['Star_Rating'] = np.nan

                try:
                    bus_info['Price'] = bus.find_element(By.CLASS_NAME, "fare.d-block").text.replace('INR ', '')
                except NoSuchElementException:
                    bus_info['Price'] = np.nan

                try:
                    seats_available_text = bus.find_element(By.CLASS_NAME, "column-eight.w-15.fl").text
                    bus_info['Seats_Available'] = seats_available_text.split()[0].strip()
                except NoSuchElementException:
                    bus_info['Seats_Available'] = np.nan

                bus_data.append(bus_info)
                print(bus_info)

            print('Bus details collected')

        except NoSuchElementException:
            print("ERROR while collecting bus elements")

    route_names = []
    route_links = []

    collect_route_details()
    
    print('Collected route names')
    print('Collected route links')


    def sroll_upto_down():
        scrolling = True
        while scrolling:
            old_page_source = driver.page_source
            body = driver.find_element(By.TAG_NAME, "body")
            for _ in range(10): 
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(1)  
            new_page_source = driver.page_source
            if new_page_source == old_page_source:
                scrolling = False


    if route_links:
        try:
            for route_name, route_link in zip(route_names, route_links):
                driver.get(route_link)
                print(f"get in to: {route_link}")
                time.sleep(5)
                next_day_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "icon.icon-right"))).click()
                time.sleep(10)
                print('Clicked next day button')
                viewbuses_button = driver.find_elements(By.CLASS_NAME, "p-left-10.icon.icon-down")
                if len(viewbuses_button) == 2: 
                    print('Found two buttons')
                    viewbuses_button2 = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="result-section"]/div[2]/div/div[2]/div/div[4]/div[2]'))).click()
                    time.sleep(10)
                    print('Clicked button 2')
                    viewbuses_button1 = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class,'clearfix srp-data')]//div//div[1]//div[1]//div[2]//div[1]//div[4]//div[2]"))).click()
                    print('Clicked button 1')
                    time.sleep(2)
                    sroll_upto_down()
                    print('Scrolled up tot down')
                    collect_bus_elements(route_name, route_link)
                    print('Collected bus elements')
                elif len(viewbuses_button) == 1:
                    print('Found only one button')
                    time.sleep(10)
                    a_viewbuses_button = wait.until(EC.element_to_be_clickable((By.XPATH,"//div[@class='button']"))).click()
                    print('Clicked only one button')
                    sroll_upto_down()
                    print('Scrolled upto down')
                    collect_bus_elements(route_name, route_link)
                    print('Collected bus elements')
                else:
                    len(viewbuses_button) == 0
                    print('Found no button')
                    time.sleep(10)
                    next_day_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "icon.icon-right"))).click()
                    print('Clicked next day button')
                    time.sleep(2)
                    sroll_upto_down()
                    print('Scrolled upto down')
                    collect_bus_elements(route_name, route_link)
                    print('Collected bus elements')
        except:
            print(f"ERROR get in to link {route_links}:")
    try:
        #convert the list  dictionaries to dataframes
        df = pd.DataFrame(bus_data)
        df.to_csv('APSRTC.csv', index=False)
        print("data saved to APSRTC.csv")
    except:
        print("ERROR WHILE CSV TO DF")

finally:
    time.sleep(21)
    driver.quit()
